In [17]:
# mandatory imports
from xcube.core.store import find_data_store_extensions
from xcube.core.store import get_data_store_params_schema
from xcube.core.store import new_data_store
from xcube.core.select import select_label_subset

# Utilities for notebook visualization
import shapely.geometry
import IPython.display
import pandas as pd
from IPython.display import JSON
import matplotlib.pyplot as plt
import math

In [18]:
sh_store = new_data_store('sentinelhub', num_retries=400)

In [19]:
sh_store.get_open_data_params_schema('S2L2A')

In [20]:
mc_def = pd.read_csv("../3_eventcoord4minicube.csv", delimiter="\t")
mc_def

,Longitude,Latitude,class
mc1,30,57,7
mc2,25,55,3
mc3,10,37,1
mc4,20,50,0
mc5,30,40,0


In [21]:
r_earth = 6378137
spatial_res_m = 20
spatial_res = 360 * spatial_res_m / (2 * math.pi * r_earth)
spatial_res

0.00017966305682390427

In [22]:
spatial_res = 0.00018
image_size = 256

In [23]:
regions = []
for row in mc_def.iterrows():
    name, (center_lon, center_lat, _) = row
    spatial_size = spatial_res * image_size / 2
    bbox = (center_lon - spatial_size,
            center_lat - spatial_size,
            center_lon + spatial_size,
            center_lat + spatial_size)
    regions.append((name, (center_lon, center_lat), bbox))

In [24]:
polygon = shapely.geometry.MultiPolygon([shapely.geometry.box(*bbox) 
                                        for _, _, bbox in regions])
IPython.display.GeoJSON(polygon.__geo_interface__)

<IPython.display.GeoJSON object>

In [9]:
s3_store = new_data_store("s3", root="deepextremes", storage_options=dict(anon=False))

In [16]:
for name, center, bbox in regions:
    mc = sh_store.open_data('S2L2A', 
                            variable_names=['B02', 'B03', 'B04', 'B06', 'B8A',  'B11', 'SCL'], 
                            bbox=bbox, 
                            spatial_res=spatial_res, 
                            time_range=('2016-01-01','2022-08-01'), 
                            time_period=None)
    out_path = f"s2_l2a_{name}_x{center[0]}_y{center[1]}.zarr"
    display(f"Writing {out_path}...")
    mc.to_zarr(f"s2_l2a_{name}_x{center[0]}_y{center[1]}.zarr", mode="w")
    # s3_store.write_data(mc, f"s2_l2a_{name}_x{center[0]}_y{center[1]}.zarr")
    display(f"Done.")    

C:\Users\norma\miniconda3\envs\xcube\lib\site-packages\pandas\core\tools\timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)


'Writing s2_l2a_mc1_x30_y57.zarr...'

'Done.'

C:\Users\norma\miniconda3\envs\xcube\lib\site-packages\pandas\core\tools\timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)


'Writing s2_l2a_mc2_x25_y55.zarr...'

...

'Writing s2_l2a_mc4_x20_y50.zarr...'

'Done.'

C:\Users\norma\miniconda3\envs\xcube\lib\site-packages\pandas\core\tools\timedeltas.py:148: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  return _coerce_scalar_to_timedelta_type(arg, unit=unit, errors=errors)


'Writing s2_l2a_mc5_x30_y40.zarr...'

'Done.'